In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
## Configure the model training policy
model_development.configure_training_policy()

In [3]:
### Load and preprocess data
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv')
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
### Prepare data for model training
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


In [5]:
### Train classification model (first phase)
optimizer = model_development.create_optimizer('nadam')
nasnetlarge = \
    model_development.nasnetlarge(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = model_development.train_classification_model(\
    training_phase = 1, model = nasnetlarge, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'NasNetLarge', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.8795 - accuracy: 0.6649 - precision: 0.6758 - recall: 0.5127
Epoch 1: val_accuracy improved from -inf to 0.59403, saving model to ../models/weights\NasNetLarge_1.00.00_fs.hdf5
119/119 [==============================] - 77s 362ms/step - loss: 0.8795 - accuracy: 0.6649 - precision: 0.6758 - recall: 0.5127 - val_loss: 0.8782 - val_accuracy: 0.5940 - val_precision: 0.5470 - val_recall: 0.8101
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5385 - accuracy: 0.7340 - precision: 0.7578 - recall: 0.6143
Epoch 2: val_accuracy improved from 0.59403 to 0.64179, saving model to ../models/weights\NasNetLarge_1.00.00_fs.hdf5
119/119 [==============================] - 38s 318ms/step - loss: 0.5385 - accuracy: 0.7340 - precision: 0.7578 - recall: 0.6143 - val_loss: 0.6496 - val_accuracy: 0.6418 - val_precision: 0.6105 - val_recall: 0.6646
Epoch 3/200
119/119 [==============================] - ETA: 0s - lo

In [6]:
## Train classification model (second phase)
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/NasNetLarge_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'NasNetLarge', version = '1.00.00_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 3.0607 - accuracy: 0.6422 - precision: 0.5975 - recall: 0.6651
Epoch 1: val_accuracy improved from -inf to 0.63881, saving model to ../models/weights\NasNetLarge_1.00.00_ss.hdf5
119/119 [==============================] - 174s 1s/step - loss: 3.0607 - accuracy: 0.6422 - precision: 0.5975 - recall: 0.6651 - val_loss: 0.7503 - val_accuracy: 0.6388 - val_precision: 0.9744 - val_recall: 0.2405
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5475 - accuracy: 0.7383 - precision: 0.8179 - recall: 0.5497
Epoch 2: val_accuracy improved from 0.63881 to 0.66866, saving model to ../models/weights\NasNetLarge_1.00.00_ss.hdf5
119/119 [==============================] - 144s 1s/step - loss: 0.5475 - accuracy: 0.7383 - precision: 0.8179 - recall: 0.5497 - val_loss: 0.6339 - val_accuracy: 0.6687 - val_precision: 0.9273 - val_recall: 0.3228
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 